# 02 · Preparación de features (activos)


In [ ]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))
from dotenv import load_dotenv
import os, pandas as pd, numpy as np

load_dotenv()

from src.utils import crear_kpis_pptos, unificar_cobertura_salud, marcar_activos

# Carpeta raíz del proyecto (sube un nivel desde /notebooks)
ROOT = Path.cwd().parent
DEFAULT = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"

# Permite override con .env si algún día quieres mover la ruta
DATA_PATH = Path(os.getenv("DATA_PATH", str(DEFAULT)))

df = pd.read_csv(DATA_PATH, low_memory=False)
pd.set_option("display.max_columns", None)  # no truncar columnas
pd.set_option("display.width", None) 

# KPIs presupuestos
df = crear_kpis_pptos(df)

# Cobertura de salud unificada
df = unificar_cobertura_salud(df)

# Marcar activos (2 años)
act_mask = marcar_activos(df, dias_umbral=730)
df_act = df[act_mask].copy()
df_inact = df[~act_mask].copy()

df_act.shape, df_inact.shape


In [ ]:
n_total = len(df)
n_act = len(df_act)
n_inact = len(df_inact)

print("Total:", n_total, " | Activos:", n_act, " | Inactivos:", n_inact)
print("Proporción activos:", round(n_act/n_total*100, 2), "%")

# Sanity check del criterio (opcional): ¿cuántos quedan activos por cada regla?
ventanas = [c for c in df.columns if c.lower().startswith('atencion')]
cualquier_atencion = df[ventanas].fillna(0).sum(axis=1) > 0
dias = pd.to_numeric(df.get('DiasDesdeUltimaVisita'), errors='coerce')
por_dias = dias.le(730).fillna(False)

print("Activos por ventanas de atención:", int(cualquier_atencion.sum()))
print("Activos por días <= 730:", int(por_dias.sum()))


## Selección de variables candidatas


In [ ]:
vars_demo = ['Edad','Sexo','Comuna','Region']
vars_kpi  = ['TicketPromPpto','PctCumplimiento','MontoAbonadoProm']
vars_act  = ['DiasDesdeUltimaVisita']
vars_att  = [c for c in df.columns if c.lower().startswith('atencion')]

vars_candidatas = vars_demo + vars_kpi + vars_act + vars_att
sorted(vars_candidatas)[:10], len(vars_candidatas)


## Limpieza mínima


In [ ]:
# Ejemplo: Sexo a binario
df_act['Sexo'] = df_act['Sexo'].map({'M':1,'F':0}).astype('Int64')

# Días desde última visita
df_act['DiasDesdeUltimaVisita'] = pd.to_numeric(df_act['DiasDesdeUltimaVisita'], errors='coerce')

# One-hot geografía (opcional, podemos hacerlo más adelante)
df_act = pd.get_dummies(df_act, columns=['Comuna','Region','CoberturaSaludCat'], drop_first=True)

df_act.shape


## Export intermedio (opcional, no subir CSV con PII)


In [ ]:
os.makedirs('data/interim', exist_ok=True)
df_act[vars_candidatas].to_csv('data/interim/clientes_activos_features.csv', index=False)
'data/interim/clientes_activos_features.csv'
